In [1]:
import os
from pathlib import Path
from dataclasses import dataclass

In [2]:
os.chdir("..")

In [3]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_file: str

In [4]:
from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from cnnClassifier.utils.common import read_yaml, create_directories

In [12]:
class ConfigManager:

    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        #self.param = read_yaml(params_filepath)
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_file=config.local_file)
        
        return data_ingestion_config


In [13]:
import os, sys
import zipfile
import gdown
from cnnClassifier.logger import logging
from cnnClassifier.utils.common import get_size
from cnnClassifier.exception import CustomException

In [14]:
class DataIngestion:

    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self):
        '''Downloads the file provided in the apprioriate url from the config data ingestion path 
        and saved it to location indicated  as per config folder 
        '''

        try:
            dataset_url = self.config.source_url
            os.makedirs(self.config.root_dir, exist_ok=True)
            zip_download_dir = os.path.join(self.config.root_dir, self.config.local_file)

            file_id = self.config.source_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            logging.info("About to start the download process")

            gdown.download(prefix+file_id,zip_download_dir)

            logging.info(f"downloaded the file at {self.config.source_url} into the file {zip_download_dir}")

        except Exception as e:
            error = CustomException(e, sys)
            logging.error(error.error_message)
            raise error

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        # make sure file is presente
        unzip_path = self.config.root_dir
        os.makedirs(unzip_path, exist_ok=True)
        path_to_unzip = os.path.join(self.config.root_dir, self.config.local_file)
        with zipfile.ZipFile(path_to_unzip, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            

In [16]:
try:
    config = ConfigManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except CustomException as e:
    error = CustomException(e, sys)
    logging.error(error.error_message)
    raise error

Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1z0mreUtRmR-P-magILsDR3T7M6IkGXtY
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1z0mreUtRmR-P-magILsDR3T7M6IkGXtY&confirm=t&uuid=e74b2b93-b270-4a34-b24f-eb6cee4e1c21
To: /Users/ngkuissi/Dev/projects/Chest-CT-Scan-Classification/artifacts/data_ingestion/data.zip
100%|██████████| 49.0M/49.0M [00:11<00:00, 4.20MB/s]
